In [2]:
! pip install yfinance


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

def calculate_metrics(actual, predicted):
    rmse = mean_squared_error(actual, predicted, squared=False)
    
    # Directional Accuracy
    actual_diff = np.diff(actual)
    predicted_diff = np.diff(predicted)
    directional_accuracy = np.sum(np.sign(actual_diff) == np.sign(predicted_diff)) / len(actual_diff)

    # Mean Average Percentage Error
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    return rmse, directional_accuracy, mape

def stock_prediction(ticker):
    # Download historical data
    data = yf.download(ticker, start="2023-01-01", end="2024-01-01")

    # Feature engineering
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    data['Volatility'] = data['Close'].rolling(window=50).std()

    # Drop rows with missing values
    data.dropna(inplace=True)

    # Define features and target
    X = data[['SMA_50', 'SMA_200', 'Volatility']]
    y = data['Close']

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train Random Forest Regressor
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions
    predictions = model.predict(X_test)

    # Calculate metrics
    rmse, directional_accuracy, mape = calculate_metrics(y_test, predictions)
    
    return rmse, directional_accuracy, mape

# List of tickers to analyze
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NFLX', 'NVDA', 'PYPL', 'INTC']

# Dictionary to store results
results = {}

for ticker in tickers:
    print(f"TICKER: {ticker}")
    rmse, directional_accuracy, mape = stock_prediction(ticker)
    results[ticker] = {'RMSE': rmse, 'Directional Accuracy': directional_accuracy, 'MAPE': mape}

# Convert results to DataFrame
results_df = pd.DataFrame(results).T

# Store results in a CSV file
results_df.to_csv('stock_predictions_metrics.csv')


TICKER: AAPL
[*********************100%%**********************]  1 of 1 completed
TICKER: MSFT
[*********************100%%**********************]  1 of 1 completed
TICKER: GOOGL
[*********************100%%**********************]  1 of 1 completed
TICKER: AMZN
[*********************100%%**********************]  1 of 1 completed
TICKER: META
[*********************100%%**********************]  1 of 1 completed
TICKER: TSLA
[*********************100%%**********************]  1 of 1 completed
TICKER: NFLX
[*********************100%%**********************]  1 of 1 completed
TICKER: NVDA
[*********************100%%**********************]  1 of 1 completed
TICKER: PYPL
[*********************100%%**********************]  1 of 1 completed
TICKER: INTC
[*********************100%%**********************]  1 of 1 completed
